In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

---

In [6]:
dsS1 = pd.read_csv('S1activities_data.csv', sep = 'delimiter', header = None, engine = 'python')

In [7]:
a = np.array(dsS1)
a = a.flatten()                                        # Flattening the 2D array to a 1D array
from more_itertools import chunked                     # Importing the required packages
a = list(chunked(a, 5))                                # Chunking the array by a unit of 5 

In [8]:
dimLength = []
i = 0
while i < len(a):
    dimLength.append(len(a[i]))
    i = i + 1

In [9]:
len(dimLength)

295

### Extracting the Sub-Activities

In [10]:
tempStringList = []                       # Temporary string list
tempStripList = []                        # Temporary 
activityList = []
dateList = []
tempStartTimeList = []
tempEndTimeList = []

i = 0
while i < len(dimLength):
    tempStringList.append(a[i][0])
    tempStripList.append([x.strip() for x in tempStringList[i].split(',')])
    activityList.append(tempStripList[i][0])
    dateList.append(tempStripList[i][1])
    tempStartTimeList.append(tempStripList[i][2])
    tempEndTimeList.append(tempStripList[i][3])
    i = i + 1

In [11]:
numericConcat = []
numericSep = []
actConcat = []
actSep = []
startConcat = []
startSep = []
endConcat = []
endSep = []

i = 0
while i < len(a):
    numericConcat.append(a[i][1])
    numericSep.append([x.strip() for x in numericConcat[i].split(',')])
    actConcat.append(a[i][2])
    actSep.append([x.strip() for x in actConcat[i].split(',')])
    startConcat.append(a[i][3])
    startSep.append([x.strip() for x in startConcat[i].split(',')])
    endConcat.append(a[i][4])
    endSep.append([x.strip() for x in endConcat[i].split(',')])
    i = i + 1

In [12]:
dateStringList = []

i = 0
while i < len(a):
    for x in range(len(numericSep[i])):
        dateStringList.append(dateList[i])
    i = i + 1

len(dateStringList)

2772

In [13]:
flat_numericSep = []                                    # Constructing an array 
for sublist in numericSep:
    for item in sublist:
        flat_numericSep.append(item)

In [14]:
flat_actSep = []
for sublist in actSep:
    for item in sublist:
        flat_actSep.append(item)

In [15]:
flat_startSep = []
for sublist in startSep:
    for item in sublist:
        flat_startSep.append(item)

In [16]:
flat_endSep = []
for sublist in endSep:
    for item in sublist:
        flat_endSep.append(item)

In [17]:
len(flat_numericSep), len(flat_actSep), len(flat_startSep), len(flat_endSep)

(2772, 2772, 2772, 2772)

In [18]:
zippedList = list(zip(flat_numericSep, flat_actSep, dateStringList, flat_startSep, flat_endSep))
ds = pd.DataFrame(zippedList, columns = ['subActNum', 'subAct', 'date', 'startTime', 'endTime'])

In [19]:
start = (ds.date + " " + ds.startTime)
end = (ds.date + " " + ds.endTime)

In [20]:
startList = []
endList = []

i = 0
while i < len(start):
    startList.append(datetime.strptime(start[i], '%m/%d/%Y %H:%M:%S'))
    endList.append(datetime.strptime(end[i], '%m/%d/%Y %H:%M:%S'))
    i = i + 1

In [21]:
zippedList = list(zip(flat_numericSep, flat_actSep, startList, endList))
ds = pd.DataFrame(zippedList, columns = ['subActNum', 'subAct', 'start', 'end'])
ds.head(n=5)

,subActNum,subAct,start,end
0,100,Toilet Flush,2003-04-01 20:51:52,2003-04-01 21:05:20
1,68,Sink faucet - hot,2003-04-01 20:51:58,2003-04-01 20:52:05
2,81,Closet,2003-04-01 20:53:36,2003-04-01 20:53:43
3,101,Light switch,2003-04-01 20:53:49,2003-04-01 21:21:43
4,93,Shower faucet,2003-04-01 20:53:52,2003-04-01 20:58:42


In [22]:
ds.head(n=4)

,subActNum,subAct,start,end
0,100,Toilet Flush,2003-04-01 20:51:52,2003-04-01 21:05:20
1,68,Sink faucet - hot,2003-04-01 20:51:58,2003-04-01 20:52:05
2,81,Closet,2003-04-01 20:53:36,2003-04-01 20:53:43
3,101,Light switch,2003-04-01 20:53:49,2003-04-01 21:21:43


In [23]:
ds['subAct'].replace('Office/study', 'Study', inplace=True)

In [24]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 4 columns):
subActNum    2772 non-null object
subAct       2772 non-null object
start        2772 non-null datetime64[ns]
end          2772 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 86.8+ KB


## Considerations for 'duplicate' sub activities
The following duplicate sensors were identified in section 1

**Create table**
<br>
3   kitchen_lightswitch <br>
4   kitchen_burner <br>
2   livingroom_lightswitch <br>
7   kitchen_drawer <br>
3   kitchen_refrigerator <br>
15  kitchen_cabinet <br>
2   kitchen_door <br>
5   bedroom_drawer <br>
2   bathroom_medicinecabinet <br>
2   bathroom_cabinet <br>

# Add explanation as to how these will be dealt with

* From `S1Sensors_preprocessed.csv` I need to find the numbers associated with these - SUBACTNUM
* 'Arbitrarily' choose one number to represent all of them
* Fill them all with that one number

**For dicussion later** - The kitchen cabinets have 15 sensors, say one of these cabinets just contains a blender, so there will always be a one-to-one between the cabinet opening and the blendor being used. This isn't particularly helpful to us. Also, more sensors could always be added... Better to do analysis with less (explain?)



In [22]:
ds.to_csv('S1SubActivities_preprocessed.csv', index = False)

---